In [10]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
# Matplotlib 폰트 설정을 위한 라이브러리 (VM 환경에서는 작동하지 않을 수 있으나, 가독성을 위해 추가)
# from matplotlib import font_manager, rc

# ----------------- 1. 데이터 로드 및 네트워크 투영 함수 -----------------
def analyze_skill_network(file_name, group_name):
    print(f"\n--- {group_name} 네트워크 분석 시작 ---")
    
    # 1. Bipartite Edge List 로드
    try:
        df_edges = pd.read_csv(file_name, encoding='utf-8')
    except Exception as e:
        print(f"❌ 오류: {file_name} 파일 로드 실패. {e}")
        return
        
    # 2. Bipartite Graph 생성
    B = nx.from_pandas_edgelist(df_edges, source='기업명', target='Skill', create_using=nx.Graph)
    
    # 3. 노드 셋 분리
    skills = set(df_edges['Skill'])
    
    # 4. One-Mode Projection (스킬-스킬 네트워크로 투영)
    # 엣지 가중치는 두 스킬을 공통으로 요구하는 기업의 수로 자동 설정됨
    G_skills = bipartite.projected_graph(B, skills)
    print(f"✅ One-Mode Skill Network 생성 완료: 노드={G_skills.number_of_nodes()}개, 엣지={G_skills.number_of_edges()}개")
    
    # 5. 중심성 계산: Degree Centrality (연결성) & PageRank (영향력)
    degree_scores = nx.degree_centrality(G_skills)
    pagerank_scores = nx.pagerank(G_skills, weight='weight', alpha=0.85)

    # 6. 결과 정렬 (Degree Centrality 기준)
    sorted_degree = sorted(degree_scores.items(), key=lambda item: item[1], reverse=True)
    
    print(f"\n--- {group_name} 핵심 스킬 (Degree Centrality 기준 Top 10) ---")
    for rank, (skill, score) in enumerate(sorted_degree[:10]):
        # PageRank 점수도 함께 출력
        pr_score = pagerank_scores.get(skill)
        print(f"순위 {rank+1}. {skill} (Degree: {score:.4f}, PR: {pr_score:.4f})")

    # 7. 네트워크 시각화
    visualize_network(G_skills, degree_scores, f'{group_name} 스킬 네트워크 (Degree Centrality)', group_name)

# ----------------- 2. 시각화 함수 정의 -----------------

def visualize_network(G, centrality_scores, title, group_name):
    plt.figure(figsize=(18, 18))
    
    # 노드 크기 설정: Degree Centrality 점수 반영 (스케일링 팩터 15000)
    node_sizes = [v * 750 for v in centrality_scores.values()]
    
    # 레이아웃 설정: Spring Layout
    pos = nx.spring_layout(G, k=2.3, iterations=50, seed=42) 
    
    # 엣지 두께 설정: 엣지 가중치(공통 요구 기업 수) 반영
    edges = G.edges(data=True)
    max_weight = max(d.get('weight', 1) for u, v, d in edges)
    # 가중치 정규화 후 두께 조정 (최대 가중치가 10의 두께를 갖도록)
    edge_widths = [d.get('weight', 1) / max_weight * 1.1 for u, v, d in edges] 

    # 엣지 그리기
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.6, width=edge_widths)
    
    # 노드와 레이블 설정 (상위 20개 노드만 레이블 표시하여 가독성 확보)
    top_nodes = sorted(centrality_scores, key=centrality_scores.get, reverse=True)[:20]
    labels = {node: node for node in top_nodes}
    
    # 노드 색상 설정 (PageRank도 고려 가능, 여기서는 통일성을 위해 Degree 사용)
    node_colors = list(centrality_scores.values())

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.YlGnBu, alpha=0.9)
    
    # 레이블 그리기 (VM 환경에서 한글 폰트 문제 발생 가능, 기본 폰트 사용)
    nx.draw_networkx_labels(G, pos, labels, font_size=10, font_color='black', verticalalignment='bottom')
    
    plt.title(title, fontsize=20)
    plt.axis('off')
    
    # 이미지 저장 (파일로 저장해야 사용자에게 전달됨)
    plt.savefig(f'{group_name}_skill_network.png')
    plt.close()
    print(f"✅ {group_name} 네트워크 시각화 파일 저장 완료: {group_name}_skill_network.png")


# ----------------- 3. 데이터 직군 분석 실행 -----------------
analyze_skill_network('C:\socialnetwork\SocialNetwork_7\SocialNetwork_Reboot\data_bipartite_skill_edges.csv', '데이터 직군')


--- 데이터 직군 네트워크 분석 시작 ---
✅ One-Mode Skill Network 생성 완료: 노드=495개, 엣지=24239개

--- 데이터 직군 핵심 스킬 (Degree Centrality 기준 Top 10) ---
순위 1. Python (Degree: 0.9049, PR: 0.0097)
순위 2. AI (Degree: 0.8482, PR: 0.0089)
순위 3. SQL (Degree: 0.7571, PR: 0.0076)
순위 4. AWS (Degree: 0.7409, PR: 0.0070)
순위 5. Docker (Degree: 0.7308, PR: 0.0070)
순위 6. Git (Degree: 0.7227, PR: 0.0070)
순위 7. Kubernetes (Degree: 0.6862, PR: 0.0063)
순위 8. CI/CD (Degree: 0.6781, PR: 0.0063)
순위 9. LLM (Degree: 0.6700, PR: 0.0066)
순위 10. API (Degree: 0.6619, PR: 0.0064)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\3184177190.py:82: UserWarning: Glyph 45936 (\N{HANGUL SYLLABLE DE}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\3184177190.py:82: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\3184177190.py:82: UserWarning: Glyph 53552 (\N{HANGUL SYLLABLE TEO}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\3184177190.py:82: UserWarning: Glyph 51649 (\N{HANGUL SYLLABLE JIG}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\3184177190.py:82: UserWarning: Glyph 44400 (\N{HANGUL SYLLABLE GUN}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png'

✅ 데이터 직군 네트워크 시각화 파일 저장 완료: 데이터 직군_skill_network.png


In [11]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
# Matplotlib 폰트 설정을 위한 라이브러리 (VM 환경에서는 작동하지 않을 수 있으나, 가독성을 위해 추가)
# from matplotlib import font_manager, rc

# ----------------- 1. 데이터 로드 및 네트워크 투영 함수 -----------------
def analyze_skill_network(file_name, group_name):
    print(f"\n--- {group_name} 네트워크 분석 시작 ---")
    
    # 1. Bipartite Edge List 로드
    try:
        df_edges = pd.read_csv(file_name, encoding='utf-8')
    except Exception as e:
        print(f"❌ 오류: {file_name} 파일 로드 실패. {e}")
        return
        
    # 2. Bipartite Graph 생성
    B = nx.from_pandas_edgelist(df_edges, source='기업명', target='Skill', create_using=nx.Graph)
    
    # 3. 노드 셋 분리
    skills = set(df_edges['Skill'])
    
    # 4. One-Mode Projection (스킬-스킬 네트워크로 투영)
    # 엣지 가중치는 두 스킬을 공통으로 요구하는 기업의 수로 자동 설정됨
    G_skills = bipartite.projected_graph(B, skills)
    print(f"✅ One-Mode Skill Network 생성 완료: 노드={G_skills.number_of_nodes()}개, 엣지={G_skills.number_of_edges()}개")
    
    # 5. 중심성 계산: Degree Centrality (연결성) & PageRank (영향력)
    degree_scores = nx.degree_centrality(G_skills)
    pagerank_scores = nx.pagerank(G_skills, weight='weight', alpha=0.85)

    # 6. 결과 정렬 (Degree Centrality 기준)
    sorted_degree = sorted(degree_scores.items(), key=lambda item: item[1], reverse=True)
    
    print(f"\n--- {group_name} 핵심 스킬 (Degree Centrality 기준 Top 10) ---")
    for rank, (skill, score) in enumerate(sorted_degree[:10]):
        # PageRank 점수도 함께 출력
        pr_score = pagerank_scores.get(skill)
        print(f"순위 {rank+1}. {skill} (Degree: {score:.4f}, PR: {pr_score:.4f})")

    # 7. 네트워크 시각화
    visualize_network(G_skills, degree_scores, f'{group_name} 스킬 네트워크 (Degree Centrality)', group_name)

# ----------------- 2. 시각화 함수 정의 -----------------

def visualize_network(G, centrality_scores, title, group_name):
    plt.figure(figsize=(18, 18))
    
    # 노드 크기 설정: Degree Centrality 점수 반영 (스케일링 팩터 15000)
    node_sizes = [v * 750 for v in centrality_scores.values()]
    
    # 레이아웃 설정: Spring Layout
    pos = nx.spring_layout(G, k=2.3, iterations=50, seed=42) 
    
    # 엣지 두께 설정: 엣지 가중치(공통 요구 기업 수) 반영
    edges = G.edges(data=True)
    max_weight = max(d.get('weight', 1) for u, v, d in edges)
    # 가중치 정규화 후 두께 조정 (최대 가중치가 10의 두께를 갖도록)
    edge_widths = [d.get('weight', 1) / max_weight * 1.1 for u, v, d in edges] 

    # 엣지 그리기
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.6, width=edge_widths)
    
    # 노드와 레이블 설정 (상위 20개 노드만 레이블 표시하여 가독성 확보)
    top_nodes = sorted(centrality_scores, key=centrality_scores.get, reverse=True)[:20]
    labels = {node: node for node in top_nodes}
    
    # 노드 색상 설정 (PageRank도 고려 가능, 여기서는 통일성을 위해 Degree 사용)
    node_colors = list(centrality_scores.values())

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.YlGnBu, alpha=0.9)
    
    # 레이블 그리기 (VM 환경에서 한글 폰트 문제 발생 가능, 기본 폰트 사용)
    nx.draw_networkx_labels(G, pos, labels, font_size=10, font_color='black', verticalalignment='bottom')
    
    plt.title(title, fontsize=20)
    plt.axis('off')
    
    # 이미지 저장 (파일로 저장해야 사용자에게 전달됨)
    plt.savefig(f'{group_name}_skill_network.png')
    plt.close()
    print(f"✅ {group_name} 네트워크 시각화 파일 저장 완료: {group_name}_skill_network.png")


# ----------------- 3. 데이터 직군 분석 실행 -----------------
analyze_skill_network('C:\socialnetwork\SocialNetwork_7\SocialNetwork_Reboot\developer_bipartite_skill_edges.csv', '개발자 직군')


--- 개발자 직군 네트워크 분석 시작 ---
✅ One-Mode Skill Network 생성 완료: 노드=688개, 엣지=56683개

--- 개발자 직군 핵심 스킬 (Degree Centrality 기준 Top 10) ---
순위 1. Git (Degree: 0.8937, PR: 0.0054)
순위 2. API (Degree: 0.8690, PR: 0.0051)
순위 3. Python (Degree: 0.8690, PR: 0.0052)
순위 4. JavaScript (Degree: 0.8690, PR: 0.0051)
순위 5. AWS (Degree: 0.8646, PR: 0.0051)
순위 6. React (Degree: 0.8574, PR: 0.0050)
순위 7. AI (Degree: 0.8544, PR: 0.0051)
순위 8. QA (Degree: 0.8297, PR: 0.0048)
순위 9. CI/CD (Degree: 0.8239, PR: 0.0048)
순위 10. Docker (Degree: 0.8137, PR: 0.0047)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\2040165149.py:82: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\2040165149.py:82: UserWarning: Glyph 48156 (\N{HANGUL SYLLABLE BAL}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\2040165149.py:82: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\2040165149.py:82: UserWarning: Glyph 51649 (\N{HANGUL SYLLABLE JIG}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.png')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\2040165149.py:82: UserWarning: Glyph 44400 (\N{HANGUL SYLLABLE GUN}) missing from font(s) DejaVu Sans.
  plt.savefig(f'{group_name}_skill_network.pn

✅ 개발자 직군 네트워크 시각화 파일 저장 완료: 개발자 직군_skill_network.png


In [16]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt

# ----------------- 1. 네트워크 투영 및 분석 함수 정의 -----------------
def analyze_bridge_skill_network(file_name, group_name):
    """
    통합 이분 그래프를 투영하고 사이중심성을 계산하여 브릿지 스킬을 도출합니다.
    """
    print(f"\n--- {group_name} 통합 네트워크 분석 시작 (사이중심성 중심) ---")
    
    # 1. Bipartite Edge List 로드
    try:
        # 통합 파일 사용
        df_edges = pd.read_csv(file_name, encoding='utf-8')
    except Exception as e:
        print(f"❌ 오류: {file_name} 파일 로드 실패. {e}")
        return
        
    # 2. Bipartite Graph 생성
    B = nx.from_pandas_edgelist(df_edges, source='기업명', target='Skill', create_using=nx.Graph)
    
    # 3. 노드 셋 분리
    skills = set(df_edges['Skill'])
    
    # 4. One-Mode Projection (스킬-스킬 네트워크로 투영)
    G_skills = bipartite.projected_graph(B, skills)
    print(f"✅ One-Mode Skill Network 생성 완료: 노드={G_skills.number_of_nodes()}개, 엣지={G_skills.number_of_edges()}개")
    
    # 5. 중심성 계산: Betweenness Centrality (사이중심성)
    # weight='weight'를 사용하여 공통 요구 빈도를 고려합니다.
    betweenness_scores = nx.betweenness_centrality(G_skills, weight='weight')

    # 6. 결과 정렬 (Betweenness Centrality 기준)
    sorted_betweenness = sorted(betweenness_scores.items(), 
                             key=lambda item: item[1], 
                             reverse=True)
    
    print(f"\n--- {group_name} Bridge Skill (사이중심성 기준 Top 15) ---")
    for rank, (skill, score) in enumerate(sorted_betweenness[:15]):
        print(f"순위 {rank+1}. {skill} (Betweenness: {score:.4f})")

    # 7. 네트워크 시각화
    visualize_network_betweenness(G_skills, betweenness_scores, f'{group_name} 통합 스킬 네트워크 (사이중심성)', 'bridge_skill_analysis')

# ----------------- 2. 시각화 함수 정의 (사이중심성 강조) -----------------

def visualize_network_betweenness(G, centrality_scores, title, file_prefix):
    """
    네트워크를 시각화하고 이미지 파일로 저장합니다.
    (노드 크기는 사이중심성에 비례, k=0.35로 거리 확보)
    """
    plt.figure(figsize=(20, 20))
    
    # 노드 크기 설정: 사이중심성 점수 반영 (스케일링 팩터 조정)
    # 이전에 효과를 보았던 1,000,000을 사용합니다.
    node_sizes = [v * 60000 for v in centrality_scores.values()]
    
    # 레이아웃 설정: Spring Layout (k=0.35로 노드 간 거리를 확보)
    pos = nx.spring_layout(G, k=2.3, iterations=50, seed=42) 
    
    # 엣지 두께 설정: 엣지 가중치 반영
    edges = G.edges(data=True)
    max_weight = max(d.get('weight', 1) for u, v, d in edges)
    edge_widths = [d.get('weight', 1) / max_weight * 3 for u, v, d in edges] 

    # 엣지 그리기
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5, width=edge_widths)
    
    # 노드와 레이블 설정 (사이중심성 Top 30개 노드만 레이블 표시)
    top_nodes = sorted(centrality_scores, key=centrality_scores.get, reverse=True)[:30]
    labels = {node: node for node in top_nodes}
    
    # 노드 색상 설정: 사이중심성 점수를 색상으로 표현 (높을수록 붉은색)
    node_colors = list(centrality_scores.values())

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.RdYlBu, alpha=0.9)
    
    # 레이블 그리기 
    nx.draw_networkx_labels(G, pos, labels, font_size=11, font_color='black', verticalalignment='bottom', font_weight='bold')
    
    plt.title(title, fontsize=24)
    plt.axis('off')
    
    # 이미지 저장
    output_filename = f'{file_prefix}_betweenness_centrality_network.png'
    plt.savefig(output_filename)
    plt.close()
    print(f"✅ 통합 네트워크 시각화 파일 저장 완료: {output_filename}")


# ----------------- 3. 통합 네트워크 분석 실행 코드 -----------------
# Merged/Bridge Group 파일을 사용합니다.
analyze_bridge_skill_network('C:\socialnetwork\SocialNetwork_7\SocialNetwork_Reboot\data&developer_bipartite_skill_edges.csv', '데이터 & 개발자 직군')


--- 데이터 & 개발자 직군 통합 네트워크 분석 시작 (사이중심성 중심) ---
✅ One-Mode Skill Network 생성 완료: 노드=728개, 엣지=71178개

--- 데이터 & 개발자 직군 Bridge Skill (사이중심성 기준 Top 15) ---
순위 1. Python (Betweenness: 0.0243)
순위 2. AI (Betweenness: 0.0204)
순위 3. Git (Betweenness: 0.0180)
순위 4. C (Betweenness: 0.0169)
순위 5. API (Betweenness: 0.0162)
순위 6. SQL (Betweenness: 0.0150)
순위 7. Docker (Betweenness: 0.0145)
순위 8. AWS (Betweenness: 0.0144)
순위 9. JavaScript (Betweenness: 0.0137)
순위 10. CI/CD (Betweenness: 0.0126)
순위 11. QA (Betweenness: 0.0118)
순위 12. React (Betweenness: 0.0115)
순위 13. Linux (Betweenness: 0.0103)
순위 14. Network (Betweenness: 0.0102)
순위 15. Kubernetes (Betweenness: 0.0101)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 45936 (\N{HANGUL SYLLABLE DE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_filename)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from font(s) DejaVu Sans.
  plt.savefig(output_filename)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 53552 (\N{HANGUL SYLLABLE TEO}) missing from font(s) DejaVu Sans.
  plt.savefig(output_filename)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.savefig(output_filename)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 48156 (\N{HANGUL SYLLABLE BAL}) missing from font(s) DejaVu Sans.
  plt.savefig(output_filename)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_27596\511442151.py:88: UserWarning: Glyph 51088 (\

✅ 통합 네트워크 시각화 파일 저장 완료: bridge_skill_analysis_betweenness_centrality_network.png
